In [50]:
import os
import tarfile
import numpy as np
from pathlib import Path
import torch
from sagemaker.local import LocalSession
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.estimator import Estimator
from sagemaker.model import Model

<method '__dir__' of 'object' objects>

In [15]:
# setting up SM local session
sagemaker_session = LocalSession()
# sagemaker_session.config = {'local': {'local_code': True}}
sagemaker_session.config = {'local': {'local_code': True }}

In [43]:
sagemaker_session.local_mode

True

In [3]:
# generating tar
with tarfile.open("model.tar.gz", mode="w:gz") as archive:
    archive.add("deploy_pkg", recursive=True)

In [62]:
# For local training a dummy role will be sufficient
base_uri = "763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.10.0-cpu-py38-ubuntu20.04-sagemaker"
role = "arn:aws:iam::687944316825:role/service-role/AmazonSageMaker-ExecutionRole-20180518T151513"
model_dir = './model.tar.gz'


# model = PyTorchModel(
#         role=role,
#         model_data=model_dir,
#         framework_version='1.7.1',
#         source_dir='code',
#         py_version='py3',
#         entry_point='inference.py'
#     )

model = PyTorchModel(
    role=role,
    model_data=model_dir,
    framework_version='1.9',
    source_dir='deploy_pkg',
    py_version='py38',
    entry_point='inference.py',
    sagemaker_session=sagemaker_session
)

In [63]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local',
)

ClientError: An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired

In [ ]:
predictor.delete_endpoint(predictor.endpoint)

In [18]:
# # image_urls: https://github.com/aws/deep-learning-containers/blob/master/available_images.md

# base = "763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.10.0-cpu-py38-ubuntu20.04-sagemaker"
# # 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.10.0-cpu-py38-ubuntu20.04-sagemaker

# estimator = Estimator(
#         image_uri=base,
#         role="arn:aws:iam::687944316825:role/service-role/AmazonSageMaker-ExecutionRole-20180518T151513",
#         instance_count=1,
#         instance_type="local",
#         input_mode="File",
#         output_path="data/",

#     )

In [57]:
# src_path = Path(Path.cwd(), 'deploy_pkg/model.tar.gz')
src_path = Path(Path.cwd(), 'opt/ml/model.tar.gz')

model = Model(
    image_uri="763104351884.dkr.ecr.eu-west-1.amazonaws.com/pytorch-inference:1.10.0-cpu-py38-ubuntu20.04-sagemaker",
    # model_data=str(src_path),
    model_data="opt/ml/model.tar.gz",
    role="arn:aws:iam::687944316825:role/service-role/AmazonSageMaker-ExecutionRole-20180518T151513",
    sagemaker_session=sagemaker_session
)


In [58]:
src_path

PosixPath('/Users/piotr.laczkowski/Desktop/PROJECTS/dex_pytorch/opt/ml/model.tar.gz')

In [59]:
model.deploy(
    initial_instance_count=1,
    instance_type="local"
)

ValueError: data_source must be either file or s3. parsed_uri.scheme: 